In [1]:
import pandas as pd
import os


In [2]:
# 1) CSV einlesen
df_raw = pd.read_csv(
    "../data/Frankenstein_Plan_Mengen_ZR.csv",
    encoding="ISO-8859-1",
    sep=";",
    header=None,
    skip_blank_lines=False,
    low_memory=False
)

# 2) Namen holen (Zeile 10 -> index 9), Einheiten (Zeile 17 -> index 16)
reihe_names = df_raw.iloc[9].astype(str).tolist()
reihe_names[0] = "von"
reihe_names[1] = "bis"
einheiten = df_raw.iloc[16].astype(str).tolist()

# 3) Daten ab Zeile 18 (index 17) extrahieren
df = df_raw.iloc[17:].copy()

# Nutze einfache Spaltenüberschriften (kein MultiIndex), um Duplikate zu vermeiden
df.columns = reihe_names
df.reset_index(drop=True, inplace=True)

# 4) Datums-Spalten NUR per Position greifen, so sind sie garantiert eindeutig
von_raw = df.iloc[:, 0].astype(str)
bis_raw = df.iloc[:, 1].astype(str)

# 5) Nur die Mess-/Zahlenspalten (ab Spalte 2) ins Float-Format bringen
num_cols = df.columns[2:]
for c in num_cols:
    # Strings bereinigen (Leerzeichen raus, Komma -> Punkt), dann zu numerisch
    df[c] = (
        df[c]
        .astype(str)
        .str.replace(r"\s+", "", regex=True)
        .str.replace(",", ".", regex=False)
        .replace({"": None})
    )
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 6) Datumsfelder sauber parsen (werden durch Schritt 5 nicht angerührt)
von = pd.to_datetime(von_raw, dayfirst=True, errors="coerce")
bis = pd.to_datetime(bis_raw, dayfirst=True, errors="coerce")

In [3]:
df.index = von

df = df.drop(columns=[("von"), ("bis")])

df

,FL_00024360.PLAN.MengeHH.2,FL_00024360.PLAN.MengeHH.1,FL_00024360.PLAN.MengeKW,FL_00024360.PLAN.MengeGA,FL_00352230.PLAN.MengeGA,FL_00025289.PLAN.MengeGA,FL_00025287.PLAN.MengeGA,FL_00025297.PLAN.MengeGA,FL_00352205.PLAN.MengeKW,FL_00352205.PLAN.MengeGA,...,CV_00349016.PLAN.Menge,CV_00349013.PLAN.Menge,CV_00349005.PLAN.Menge,CV_00349004.PLAN.Menge,CV_00349002.PLAN.Menge,CV_00348857.PLAN.Menge,KN_00005030.PLAN.Menge.2,KN_00005030.PLAN.Menge.1,KN_00005023.PLAN.Menge.2,KN_00005023.PLAN.Menge.1
von,,,,,,,,,,,,,,,,,,,,,
2014-01-01 06:00:00,51.476,1709.224,461.161,1025.914,45.360,28.190,57.864,121.684,0.000,72.563,...,NaN,NaN,NaN,0.753,34.045,NaN,19.550,1.079,7.284,1.900
2014-01-01 07:00:00,51.026,1671.588,453.456,1022.869,42.560,28.224,57.765,117.911,0.000,76.735,...,NaN,NaN,NaN,0.607,35.279,NaN,18.923,0.955,6.890,1.562
2014-01-01 08:00:00,50.688,1817.313,454.455,1027.474,64.546,27.976,60.100,120.903,0.000,67.399,...,NaN,NaN,NaN,0.573,32.063,NaN,18.832,0.865,6.396,1.664
2014-01-01 09:00:00,50.857,1725.431,455.622,1022.988,72.453,28.134,60.483,120.126,0.000,65.328,...,NaN,NaN,NaN,0.495,28.629,NaN,18.636,0.865,6.890,1.619
2014-01-01 10:00:00,50.873,1997.502,454.479,1024.464,69.048,27.853,60.366,120.459,0.000,59.546,...,NaN,NaN,NaN,0.562,28.563,NaN,18.158,0.787,8.497,1.630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 01:00:00,0.000,862.281,25.255,258.222,31.612,33.446,37.185,164.931,6.775,216.890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 02:00:00,0.000,789.112,27.725,253.316,35.805,33.605,38.159,165.078,8.112,196.111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 03:00:00,0.000,791.423,28.767,245.283,33.190,33.616,39.938,155.266,6.050,188.599,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from utils import load_series_dfs
series_dfs = load_series_dfs(data_dir=r"C:\Users\johan\Documents\FH_Master\data", filename="final_learning_dfs.pkl")
series_dfs['FL_00024702.PLAN.MengeHH'].columns

Index(['consumption', 'hour', 'weekday', 'month', 'is_weekend', 'w_tl', 'w_rf',
       'w_ff', 'w_ffx', 'w_cglo', 'w_so_h', 'w_rr', 'w_rrm', 'w_tb10',
       'w_tb20', 'CEGH_WAP', 'THE_WAP'],
      dtype='object')

In [5]:
series_dfs['FL_00024702.PLAN.MengeHH']

,consumption,hour,weekday,month,is_weekend,w_tl,w_rf,w_ff,w_ffx,w_cglo,w_so_h,w_rr,w_rrm,w_tb10,w_tb20,CEGH_WAP,THE_WAP
von | von,,,,,,,,,,,,,,,,,
2015-01-01 06:00:00,48.447,6,3,1,0,-0.2,92.0,0.2,2.1,0.0,0.0,0.0,5.0,2.8,2.9,23.61,21.50
2015-01-01 07:00:00,45.788,7,3,1,0,0.0,94.0,0.2,1.5,0.0,0.0,0.0,0.0,2.8,2.9,23.61,21.50
2015-01-01 08:00:00,43.109,8,3,1,0,0.3,93.0,0.6,4.0,8.0,0.0,0.0,5.0,2.9,2.9,23.61,21.50
2015-01-01 09:00:00,41.749,9,3,1,0,0.8,90.0,0.4,3.0,53.0,0.0,0.0,0.0,2.9,2.9,23.61,21.50
2015-01-01 10:00:00,39.900,10,3,1,0,1.1,88.0,0.3,3.7,97.0,0.0,0.0,0.0,2.9,2.9,23.61,21.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,32.223,19,0,12,0,3.9,95.0,0.6,3.2,0.0,0.0,0.0,0.0,4.0,3.3,23.10,23.12
2018-12-31 20:00:00,29.348,20,0,12,0,3.9,96.0,0.3,3.4,0.0,0.0,0.0,0.0,4.0,3.3,23.10,23.12
2018-12-31 21:00:00,24.727,21,0,12,0,3.9,96.0,0.5,2.6,0.0,0.0,0.0,2.0,4.0,3.3,23.10,23.12


In [6]:
df.columns

Index(['FL_00024360.PLAN.MengeHH.2', 'FL_00024360.PLAN.MengeHH.1',
       'FL_00024360.PLAN.MengeKW', 'FL_00024360.PLAN.MengeGA',
       'FL_00352230.PLAN.MengeGA', 'FL_00025289.PLAN.MengeGA',
       'FL_00025287.PLAN.MengeGA', 'FL_00025297.PLAN.MengeGA',
       'FL_00352205.PLAN.MengeKW', 'FL_00352205.PLAN.MengeGA',
       ...
       'CV_00349016.PLAN.Menge', 'CV_00349013.PLAN.Menge',
       'CV_00349005.PLAN.Menge', 'CV_00349004.PLAN.Menge',
       'CV_00349002.PLAN.Menge', 'CV_00348857.PLAN.Menge',
       'KN_00005030.PLAN.Menge.2', 'KN_00005030.PLAN.Menge.1',
       'KN_00005023.PLAN.Menge.2', 'KN_00005023.PLAN.Menge.1'],
      dtype='object', length=322)

In [7]:
   df.index.max(), df.index.min()

(Timestamp('2022-01-01 05:00:00'), Timestamp('2014-01-01 06:00:00'))

In [8]:
series_dfs['FL_00024702.PLAN.MengeHH'].index.max() , series_dfs['FL_00024702.PLAN.MengeHH'].index.min()

(Timestamp('2018-12-31 23:00:00'), Timestamp('2015-01-01 06:00:00'))

In [9]:
# --- 1) Testzeitraum definieren ---
end_date = "2018-12-31 23:00:00"

start_date = '2015-01-01 06:00:00'
df_sub = df.loc[start_date:end_date]

# --- 2) Anteil fehlender Werte pro Zeitreihe berechnen ---
na_ratio = df_sub.isna().mean()

# --- 3) Bins definieren ---
bins = [0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 1.0]
labels = [
    "0–5%", "5–10%", "10–15%", "15–20%", "20–25%", "25–30%", ">30%"
]

# --- 4) Einteilung in Kategorien ---
categories = pd.cut(na_ratio, bins=bins, labels=labels, include_lowest=True)

# --- 5) Ergebnis-DataFrame ---
na_overview = pd.DataFrame({
    "na_ratio": na_ratio,
    "bin": categories
}).sort_values("na_ratio", ascending=False)

# --- 6) Zugriff auf einzelne Bins ---
cols_10_15 = na_overview[na_overview["bin"] == "10–15%"].index.tolist()
cols_15_20 = na_overview[na_overview["bin"] == "15–20%"].index.tolist()
cols_20_25 = na_overview[na_overview["bin"] == "20–25%"].index.tolist()
cols_25_30 = na_overview[na_overview["bin"] == "25–30%"].index.tolist()

print(na_overview["bin"].value_counts())

bin
10–15%    133
0–5%       63
20–25%     62
5–10%      36
>30%       27
25–30%      1
15–20%      0
Name: count, dtype: int64


In [10]:
na_overview.bin.unique()

['>30%', '25–30%', '20–25%', '10–15%', '5–10%', '0–5%']
Categories (7, object): ['0–5%' < '5–10%' < '10–15%' < '15–20%' < '20–25%' < '25–30%' < '>30%']

In [11]:
len(na_overview[na_overview["bin"] == '5–10%']), len(na_overview[na_overview["bin"] == '10–15%'])

(36, 133)

In [46]:
na_overview[na_overview["bin"] == '10–15%'].loc['KN_00005132.PLAN.Menge']

na_ratio                                        0.104598
bin                                               10–15%
nemesys_key                                  KN_00005132
geom_g         POINT (522764.076656527 5218206.57699626)
geometry       POINT (522764.076656527 5218206.57699626)
pt_interior    POINT (522764.076656527 5218206.57699626)
pt_x                                       522764.076657
pt_y                                      5218206.576996
Name: KN_00005132.PLAN.Menge, dtype: object

In [51]:
df_sub['KN_00005132.PLAN.Menge'].isna().sum()

np.int64(3667)

In [13]:
csv_path = os.path.join('..', 'data', 'gdm_ausspp_202505011423.csv')

# 2. Laden
df_knoten = pd.read_csv(csv_path)
df1 = df_knoten[['nemesys_key', 'geom_g']]

C:\Users\johan\AppData\Local\Temp\ipykernel_28224\3073625327.py:4: DtypeWarning: Columns (18,19,31,32,33,34,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_knoten = pd.read_csv(csv_path)


In [14]:
csv_path = os.path.join('..', 'data', 'gdm_mengem_202505011301.csv')

# 2. Laden
df_punkte = pd.read_csv(csv_path)
df2 = df_punkte[['nemesys_key', 'geom_g']]

In [15]:
csv_path = os.path.join('..', 'data', 'gdm_verbz_202505011429.csv')

# 2. Laden
df_flaechen = pd.read_csv(csv_path)
df3 = df_flaechen[['nemesys_key', 'geom_g']]

In [16]:
frames = [df1, df2, df3]

result = pd.concat(frames)
len(result.index), (len(df1.index) + len(df2.index) + len(df3.index))

(58413, 58413)

In [17]:
result.head()

,nemesys_key,geom_g
0,KN_00370061,POINT (467432.38972410024 5298251.233962588)
1,KN_00400891,POINT (638160.7506623925 5317827.993315005)
2,KN_00400949,POINT (639183.4338244994 5316622.424836051)
3,KN_00397400,POINT (638188.4998389793 5317801.508679103)
4,KN_00397666,POINT (638206.87598522 5317780.619306837)


In [18]:
# 1) Prefix-Key aus dem na_overview-Index ableiten (alles vor dem ersten ".")
na_overview = na_overview.copy()
na_overview["nemesys_key"] = (
    pd.Index(na_overview.index)
      .to_series()
      .str.split(".", n=1, expand=True)[0]
)

# 2) Eindeutige Geometrien je nemesys_key vorbereiten (bei Duplikaten die erste behalten)
geom_map = (
    result
    .drop_duplicates(subset=["nemesys_key"])
    .set_index("nemesys_key")["geom_g"]
)

# 3) Geometrie in na_overview mappen
na_overview["geom_g"] = na_overview["nemesys_key"].map(geom_map)

# 4) Kurz-Check & Diagnostics
total = len(na_overview)
matched = na_overview["geom_g"].notna().sum()
unmatched_keys = (
    na_overview.loc[na_overview["geom_g"].isna(), "nemesys_key"]
    .nunique()
)

print(f"Gematchte Reihen: {matched}/{total} ({matched/total:.1%})")
print(f"Anzahl unterschiedlicher nemesys_keys ohne Geometrie: {unmatched_keys}")

Gematchte Reihen: 319/322 (99.1%)
Anzahl unterschiedlicher nemesys_keys ohne Geometrie: 2


In [19]:
na_overview

,na_ratio,bin,nemesys_key,geom_g
FL_00352208.PLAN.MengeKW,1.0,>30%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...
KN_00003623.PLAN.Menge,1.0,>30%,KN_00003623,POINT (629249.710253795 5319336.72648159)
KN_00003613.PLAN.Menge,1.0,>30%,KN_00003613,POINT (619503.043571605 5315717.57029582)
KN_00003602.PLAN.Menge,1.0,>30%,KN_00003602,POINT (616169.6906068642 5315623.142645395)
KN_00003549.PLAN.Menge,1.0,>30%,KN_00003549,POINT (613600.973378887 5316674.7537472)
...,...,...,...,...
FL_00352208.PLAN.MengeGA,0.0,0–5%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...
FL_00025248.PLAN.MengeGA,0.0,0–5%,FL_00025248,MULTIPOLYGON (((571359.582255836 5379201.19160...
FL_00352054.PLAN.MengeHH,0.0,0–5%,FL_00352054,MULTIPOLYGON (((168517.508101939 5271056.46775...
CV_00348266.PLAN.Menge,0.0,0–5%,CV_00348266,"LINESTRING (392147.529330797 5313778.26758383,..."


In [20]:
import requests
# 1. Resource-ID für stündliche Klimastationen
resource_id = 'klima-v2-1h'

# 2. Metadata-Endpoint ansprechen
url_meta  = f"https://dataset.api.hub.geosphere.at/v1/station/historical/{resource_id}/metadata"
resp_meta = requests.get(url_meta)
resp_meta.raise_for_status()

# 3. Liste der Station-Dicts rausziehen
data = resp_meta.json()          # {'stations': [ {...}, {...}, … ]}
stations_list = data['stations']

# 4. In DataFrame umwandeln
stations_df = pd.DataFrame.from_records(stations_list)

# 5. Wichtige Spalten anzeigen
print("Spalten:", stations_df.columns.tolist())
print(stations_df[['id','name','lat','lon']].head())

Spalten: ['type', 'id', 'group_id', 'name', 'state', 'lat', 'lon', 'altitude', 'valid_from', 'valid_to', 'has_sunshine', 'has_global_radiation', 'is_active']
   id              name       lat       lon
0   1            Aflenz  47.54594  15.24069
1   2  Aigen im Ennstal  47.53278  14.13826
2   3       Allentsteig  48.69083  15.36694
3   4         Amstetten  48.10889  14.89500
4   5        Bad Aussee  47.61050  13.75844


In [21]:
df_weather = pd.read_csv('../data/wetterdaten_top150_filled_linear.csv.gz', compression='gzip', parse_dates=['timestamp'])
df_weather.head()

stations_df = stations_df.query("id in @df_weather.station_id.unique()")


In [22]:
stations_df

,type,id,group_id,name,state,lat,lon,altitude,valid_from,valid_to,has_sunshine,has_global_radiation,is_active
1,COMBINED,2,NaN,Aigen im Ennstal,Steiermark,47.532780,14.13826,641.0,1993-02-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
2,COMBINED,3,NaN,Allentsteig,Niederösterreich,48.690830,15.36694,598.8,1992-10-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
6,COMBINED,7,NaN,Bad Gastein,Salzburg,47.110560,13.13336,1092.0,1926-04-21T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
12,COMBINED,13,NaN,Bernstein,Burgenland,47.408330,16.26139,630.8,2008-01-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
17,COMBINED,18,NaN,Bruckneudorf,Burgenland,48.012780,16.84500,165.6,2008-01-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,INDIVIDUAL,19911,NaN,Hermagor,Kärnten,46.609720,13.49222,561.6,2008-07-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
778,INDIVIDUAL,20101,122.0,Kanzelhöhe,Kärnten,46.677220,13.90194,1520.0,2002-01-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
793,INDIVIDUAL,20212,48.0,Klagenfurt Flughafen,Kärnten,46.648330,14.31833,450.0,1996-10-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True
801,INDIVIDUAL,20270,NaN,Feldkirchen,Kärnten,46.721940,14.09694,546.4,1995-07-01T00:00:00+00:00,2100-12-31T00:00:00+00:00,True,True,True


In [23]:
import pandas as pd
import geopandas as gpd
from shapely import wkt as shapely_wkt

# 1) WKT -> Shapely-Geometrien (robust)
def parse_geom(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    if hasattr(x, "geom_type"):            # bereits Geometrie
        return x
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return None
        try:
            return shapely_wkt.loads(s)
        except Exception:
            return None
    return None

geoms = na_overview["geom_g"].apply(parse_geom)

# 2) GeoDataFrame mit richtigem CRS (UTM 33N)
gdf = gpd.GeoDataFrame(na_overview.copy(), geometry=geoms, crs="EPSG:32633")

# 3) Interior-Point / Repräsentativpunkt sicher bilden
def interior_point_safe(g):
    if g is None or g.is_empty:
        return None
    t = g.geom_type
    if t in ("Polygon", "MultiPolygon"):
        return g.representative_point()
    elif t in ("LineString", "MultiLineString"):
        try:
            return g.interpolate(0.5, normalized=True)
        except Exception:
            return g.centroid
    elif t in ("Point", "MultiPoint"):
        return g.centroid
    else:
        return g.centroid

gdf["pt_interior"] = gdf.geometry.apply(interior_point_safe)
gdf["pt_x"] = gdf["pt_interior"].x
gdf["pt_y"] = gdf["pt_interior"].y

# zurückschreiben, wenn du mit na_overview weiterarbeiten willst
na_overview = gdf

# Diagnostics (optional)
print("Geometrien:", na_overview.geometry.notna().sum(), "/", len(na_overview))
print("Interior-Points:", na_overview["pt_interior"].notna().sum())


Geometrien: 319 / 322
Interior-Points: 319


In [24]:
na_overview

,na_ratio,bin,nemesys_key,geom_g,geometry,pt_interior,pt_x,pt_y
FL_00352208.PLAN.MengeKW,1.0,>30%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,"MULTIPOLYGON (((453137.43 5163557.527, 453137....",POINT (449581.692 5173939.372),449581.692322,5.173939e+06
KN_00003623.PLAN.Menge,1.0,>30%,KN_00003623,POINT (629249.710253795 5319336.72648159),POINT (629249.71 5319336.726),POINT (629249.71 5319336.726),629249.710254,5.319337e+06
KN_00003613.PLAN.Menge,1.0,>30%,KN_00003613,POINT (619503.043571605 5315717.57029582),POINT (619503.044 5315717.57),POINT (619503.044 5315717.57),619503.043572,5.315718e+06
KN_00003602.PLAN.Menge,1.0,>30%,KN_00003602,POINT (616169.6906068642 5315623.142645395),POINT (616169.691 5315623.143),POINT (616169.691 5315623.143),616169.690607,5.315623e+06
KN_00003549.PLAN.Menge,1.0,>30%,KN_00003549,POINT (613600.973378887 5316674.7537472),POINT (613600.973 5316674.754),POINT (613600.973 5316674.754),613600.973379,5.316675e+06
...,...,...,...,...,...,...,...,...
FL_00352208.PLAN.MengeGA,0.0,0–5%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,"MULTIPOLYGON (((453137.43 5163557.527, 453137....",POINT (449581.692 5173939.372),449581.692322,5.173939e+06
FL_00025248.PLAN.MengeGA,0.0,0–5%,FL_00025248,MULTIPOLYGON (((571359.582255836 5379201.19160...,"MULTIPOLYGON (((571359.582 5379201.192, 571359...",POINT (579850.461 5385507.515),579850.461225,5.385508e+06
FL_00352054.PLAN.MengeHH,0.0,0–5%,FL_00352054,MULTIPOLYGON (((168517.508101939 5271056.46775...,"MULTIPOLYGON (((168517.508 5271056.468, 168586...",POINT (178961.588 5264391.467),178961.587808,5.264391e+06
CV_00348266.PLAN.Menge,0.0,0–5%,CV_00348266,"LINESTRING (392147.529330797 5313778.26758383,...","LINESTRING (392147.529 5313778.268, 392158.254...",POINT (392152.892 5313778.267),392152.891582,5.313778e+06


In [25]:
import geopandas as gpd
df_pts = gpd.GeoDataFrame(
    na_overview.drop(columns='geometry'),  # alte geometry-Spalte entfernen, falls gewünscht
    geometry=na_overview['pt_interior'],
    crs="EPSG:32633"
)

# 3. Jetzt transformierst Du df_pts in WGS84
df_pts = df_pts.to_crs(epsg=4326)

# 4. Longitude / Latitude aus der neuen Geometrie ziehen
df_pts['centroid_lon'] = df_pts.geometry.x
df_pts['centroid_lat'] = df_pts.geometry.y

# 5. Kontrolle
print(df_pts[['nemesys_key','centroid_lon','centroid_lat']].head())

                          nemesys_key  centroid_lon  centroid_lat
FL_00352208.PLAN.MengeKW  FL_00352208     14.340297     46.717107
KN_00003623.PLAN.Menge    KN_00003623     16.733179     48.014244
KN_00003613.PLAN.Menge    KN_00003613     16.601528     47.983592
KN_00003602.PLAN.Menge    KN_00003602     16.556847     47.983357
KN_00003549.PLAN.Menge    KN_00003549     16.522714     47.993276


In [26]:
df_pts.dropna(inplace=True)
df_pts

,na_ratio,bin,nemesys_key,geom_g,pt_interior,pt_x,pt_y,geometry,centroid_lon,centroid_lat
FL_00352208.PLAN.MengeKW,1.0,>30%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,POINT (449581.692 5173939.372),449581.692322,5.173939e+06,POINT (14.3403 46.71711),14.340297,46.717107
KN_00003623.PLAN.Menge,1.0,>30%,KN_00003623,POINT (629249.710253795 5319336.72648159),POINT (629249.71 5319336.726),629249.710254,5.319337e+06,POINT (16.73318 48.01424),16.733179,48.014244
KN_00003613.PLAN.Menge,1.0,>30%,KN_00003613,POINT (619503.043571605 5315717.57029582),POINT (619503.044 5315717.57),619503.043572,5.315718e+06,POINT (16.60153 47.98359),16.601528,47.983592
KN_00003602.PLAN.Menge,1.0,>30%,KN_00003602,POINT (616169.6906068642 5315623.142645395),POINT (616169.691 5315623.143),616169.690607,5.315623e+06,POINT (16.55685 47.98336),16.556847,47.983357
KN_00003549.PLAN.Menge,1.0,>30%,KN_00003549,POINT (613600.973378887 5316674.7537472),POINT (613600.973 5316674.754),613600.973379,5.316675e+06,POINT (16.52271 47.99328),16.522714,47.993276
...,...,...,...,...,...,...,...,...,...,...
FL_00352208.PLAN.MengeGA,0.0,0–5%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,POINT (449581.692 5173939.372),449581.692322,5.173939e+06,POINT (14.3403 46.71711),14.340297,46.717107
FL_00025248.PLAN.MengeGA,0.0,0–5%,FL_00025248,MULTIPOLYGON (((571359.582255836 5379201.19160...,POINT (579850.461 5385507.515),579850.461225,5.385508e+06,POINT (16.08345 48.61754),16.083447,48.617541
FL_00352054.PLAN.MengeHH,0.0,0–5%,FL_00352054,MULTIPOLYGON (((168517.508101939 5271056.46775...,POINT (178961.588 5264391.467),178961.587808,5.264391e+06,POINT (10.74071 47.45377),10.740705,47.453774
CV_00348266.PLAN.Menge,0.0,0–5%,CV_00348266,"LINESTRING (392147.529330797 5313778.26758383,...",POINT (392152.892 5313778.267),392152.891582,5.313778e+06,POINT (13.55511 47.96822),13.555113,47.968221


In [27]:
import numpy as np
from sklearn.neighbors import BallTree
import pandas as pd

# 1. Stations-Koordinaten in Radianten
coords_stat = np.deg2rad(stations_df[['lat','lon']].values)

# 2. Centroid-Interior-Koordinaten in Radianten
coords_map  = np.deg2rad(df_pts[['centroid_lat','centroid_lon']].values)

# 3. BallTree mit Haversine-Metrik bauen
tree = BallTree(coords_stat, metric='haversine')

# 4. Für jeden Centroid den nächsten Nachbarn suchen
dist_rad, idx = tree.query(coords_map, k=1)

# 5. In Kilometern umrechnen
dist_km = dist_rad.flatten() * 6371.0

# 6. Ergebnis-Tabelle zusammenbauen
result = pd.DataFrame({
    'nemesys_key':  df_pts['nemesys_key'].values,
    'stat_id':      stations_df.iloc[idx.flatten()]['id'].values,
    'stat_name':    stations_df.iloc[idx.flatten()]['name'].values,
    'dist_km':      dist_km
})

print(result.head())

   nemesys_key  stat_id            stat_name   dist_km
0  FL_00352208       95  St.Veit an der Glan  4.233229
1  KN_00003623     6102         Bruckneudorf  8.319344
2  KN_00003613     7710          Seibersdorf  7.289190
3  KN_00003602     7710          Seibersdorf  4.009748
4  KN_00003549     7710          Seibersdorf  2.445193


In [54]:
df_weather = pd.read_csv('../data/wetterdaten_top150_filled_linear.csv.gz', compression='gzip', parse_dates=['timestamp'])

# 1) Konsistenz: timestamp -> UTC-aware
df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"], utc=True)

# 2) Cutoff als UTC-aware definieren
CUTOFF = pd.Timestamp("2015-01-01 06:00:00", tz="UTC")

# 3) Filtern ab Cutoff
df_weather = df_weather[df_weather["timestamp"] >= CUTOFF].copy()

# 4) Für den Merge: station_id == stat_id (umbenennen)
df_weather = df_weather.rename(columns={"station_id": "stat_id"})
df_weather.head()
df_weather.head()

,timestamp,station_id,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
0,2014-01-01 00:00:00+00:00,2,-2.7,93.0,1.7,2.6,0.0,0.0,0.0,0.0,0.5,1.1
1,2014-01-01 00:00:00+00:00,7604,4.0,84.0,3.2,6.3,0.0,0.0,0.0,0.0,3.9,5.2
2,2014-01-01 00:00:00+00:00,48,1.5,88.0,0.9,2.2,0.0,0.0,0.0,0.0,3.0,3.7
3,2014-01-01 00:00:00+00:00,20412,1.4,100.0,0.2,1.5,0.0,0.0,0.0,0.0,2.8,3.2
4,2014-01-01 00:00:00+00:00,89,1.5,95.0,0.7,3.3,0.0,0.0,0.0,0.0,2.1,2.2


In [29]:
df.head()

,FL_00024360.PLAN.MengeHH.2,FL_00024360.PLAN.MengeHH.1,FL_00024360.PLAN.MengeKW,FL_00024360.PLAN.MengeGA,FL_00352230.PLAN.MengeGA,FL_00025289.PLAN.MengeGA,FL_00025287.PLAN.MengeGA,FL_00025297.PLAN.MengeGA,FL_00352205.PLAN.MengeKW,FL_00352205.PLAN.MengeGA,...,CV_00349016.PLAN.Menge,CV_00349013.PLAN.Menge,CV_00349005.PLAN.Menge,CV_00349004.PLAN.Menge,CV_00349002.PLAN.Menge,CV_00348857.PLAN.Menge,KN_00005030.PLAN.Menge.2,KN_00005030.PLAN.Menge.1,KN_00005023.PLAN.Menge.2,KN_00005023.PLAN.Menge.1
von,,,,,,,,,,,,,,,,,,,,,
2014-01-01 06:00:00,51.476,1709.224,461.161,1025.914,45.360,28.190,57.864,121.684,0.0,72.563,...,NaN,NaN,NaN,0.753,34.045,NaN,19.550,1.079,7.284,1.900
2014-01-01 07:00:00,51.026,1671.588,453.456,1022.869,42.560,28.224,57.765,117.911,0.0,76.735,...,NaN,NaN,NaN,0.607,35.279,NaN,18.923,0.955,6.890,1.562
2014-01-01 08:00:00,50.688,1817.313,454.455,1027.474,64.546,27.976,60.100,120.903,0.0,67.399,...,NaN,NaN,NaN,0.573,32.063,NaN,18.832,0.865,6.396,1.664
2014-01-01 09:00:00,50.857,1725.431,455.622,1022.988,72.453,28.134,60.483,120.126,0.0,65.328,...,NaN,NaN,NaN,0.495,28.629,NaN,18.636,0.865,6.890,1.619
2014-01-01 10:00:00,50.873,1997.502,454.479,1024.464,69.048,27.853,60.366,120.459,0.0,59.546,...,NaN,NaN,NaN,0.562,28.563,NaN,18.158,0.787,8.497,1.630


In [30]:
df_price = pd.read_excel('../data/gas_price.xlsx')

df_price = df_price.set_index('Datum')


df_price = df_price.sort_index()

df_price.head()

,CEGHIX,TRP NCG
Datum,,
2015-01-01 06:00:00,23.61,21.5
2015-01-01 07:00:00,23.61,21.5
2015-01-01 08:00:00,23.61,21.5
2015-01-01 09:00:00,23.61,21.5
2015-01-01 10:00:00,23.61,21.5


In [31]:
result.head()

,nemesys_key,stat_id,stat_name,dist_km
0,FL_00352208,95,St.Veit an der Glan,4.233229
1,KN_00003623,6102,Bruckneudorf,8.319344
2,KN_00003613,7710,Seibersdorf,7.289190
3,KN_00003602,7710,Seibersdorf,4.009748
4,KN_00003549,7710,Seibersdorf,2.445193


In [32]:
df_pts.head()

,na_ratio,bin,nemesys_key,geom_g,pt_interior,pt_x,pt_y,geometry,centroid_lon,centroid_lat
FL_00352208.PLAN.MengeKW,1.0,>30%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,POINT (449581.692 5173939.372),449581.692322,5.173939e+06,POINT (14.3403 46.71711),14.340297,46.717107
KN_00003623.PLAN.Menge,1.0,>30%,KN_00003623,POINT (629249.710253795 5319336.72648159),POINT (629249.71 5319336.726),629249.710254,5.319337e+06,POINT (16.73318 48.01424),16.733179,48.014244
KN_00003613.PLAN.Menge,1.0,>30%,KN_00003613,POINT (619503.043571605 5315717.57029582),POINT (619503.044 5315717.57),619503.043572,5.315718e+06,POINT (16.60153 47.98359),16.601528,47.983592
KN_00003602.PLAN.Menge,1.0,>30%,KN_00003602,POINT (616169.6906068642 5315623.142645395),POINT (616169.691 5315623.143),616169.690607,5.315623e+06,POINT (16.55685 47.98336),16.556847,47.983357
KN_00003549.PLAN.Menge,1.0,>30%,KN_00003549,POINT (613600.973378887 5316674.7537472),POINT (613600.973 5316674.754),613600.973379,5.316675e+06,POINT (16.52271 47.99328),16.522714,47.993276


In [52]:
df_pts2 = df_pts[['bin', 'nemesys_key']]
df_pts2.head()

,bin,nemesys_key
FL_00352208.PLAN.MengeKW,>30%,FL_00352208
KN_00003623.PLAN.Menge,>30%,KN_00003623
KN_00003613.PLAN.Menge,>30%,KN_00003613
KN_00003602.PLAN.Menge,>30%,KN_00003602
KN_00003549.PLAN.Menge,>30%,KN_00003549


In [53]:
df_sub.head()

,FL_00024360.PLAN.MengeHH.2,FL_00024360.PLAN.MengeHH.1,FL_00024360.PLAN.MengeKW,FL_00024360.PLAN.MengeGA,FL_00352230.PLAN.MengeGA,FL_00025289.PLAN.MengeGA,FL_00025287.PLAN.MengeGA,FL_00025297.PLAN.MengeGA,FL_00352205.PLAN.MengeKW,FL_00352205.PLAN.MengeGA,...,CV_00349016.PLAN.Menge,CV_00349013.PLAN.Menge,CV_00349005.PLAN.Menge,CV_00349004.PLAN.Menge,CV_00349002.PLAN.Menge,CV_00348857.PLAN.Menge,KN_00005030.PLAN.Menge.2,KN_00005030.PLAN.Menge.1,KN_00005023.PLAN.Menge.2,KN_00005023.PLAN.Menge.1
von,,,,,,,,,,,,,,,,,,,,,
2015-01-01 06:00:00,51.213,2513.079,989.685,966.096,46.055,32.710,49.957,206.050,0.0,209.432,...,NaN,NaN,NaN,0.912,55.417,NaN,21.040,1.272,10.900,2.781
2015-01-01 07:00:00,51.302,2567.940,990.999,985.808,47.439,35.390,54.402,202.630,0.0,207.655,...,NaN,NaN,NaN,0.878,52.402,NaN,20.767,1.182,10.663,2.601
2015-01-01 08:00:00,51.568,2439.627,944.198,980.405,45.104,34.838,54.263,200.787,0.0,209.411,...,NaN,NaN,NaN,0.799,49.213,NaN,20.492,1.137,9.177,2.567
2015-01-01 09:00:00,52.012,2428.044,997.779,943.864,42.664,37.124,50.678,204.838,0.0,219.783,...,NaN,NaN,NaN,0.754,47.489,NaN,20.005,1.092,9.391,2.590
2015-01-01 10:00:00,51.745,2511.695,1005.459,880.828,40.548,32.654,52.588,207.463,0.0,204.083,...,NaN,NaN,NaN,0.698,48.061,NaN,19.702,1.002,8.625,2.534


In [59]:

# Cutoff definieren



,timestamp,stat_id,tl,rf,ff,ffx,cglo,so_h,rr,rrm,tb10,tb20
1314900,2015-01-01 06:00:00+00:00,66,-6.0,86.0,0.3,2.2,0.0,0.0,0.0,0.0,0.0,0.2
1314901,2015-01-01 06:00:00+00:00,20412,-7.5,90.0,0.6,1.4,0.0,0.0,0.0,0.0,-1.2,0.0
1314902,2015-01-01 06:00:00+00:00,17301,-6.0,86.0,0.3,2.2,0.0,0.0,0.0,0.0,0.0,0.2
1314903,2015-01-01 06:00:00+00:00,103,-9.4,92.0,1.0,2.2,0.0,0.0,0.0,0.0,0.0,1.5
1314904,2015-01-01 06:00:00+00:00,5735,-1.4,97.0,11.1,17.8,0.0,0.0,0.0,29.0,1.5,2.9


In [66]:
import pandas as pd

# ================================
# 0) Hilfsfunktionen
# ================================
def ensure_utc_index(df, assume_col_name=None):
    """
    Stellt sicher, dass df einen DatetimeIndex in UTC hat.
    - Wenn 'assume_col_name' angegeben und als Spalte vorhanden ist, wird diese Spalte zum Index.
    - Andernfalls wird der bestehende Index konvertiert.
    """
    out = df.copy()

    if assume_col_name and assume_col_name in out.columns:
        out[assume_col_name] = pd.to_datetime(out[assume_col_name], errors="coerce", utc=True)
        out = out.dropna(subset=[assume_col_name]).set_index(assume_col_name)
    else:
        # vorhandenen Index verwenden
        idx = pd.to_datetime(out.index, errors="coerce")
        if getattr(idx, "tz", None) is None:
            idx = idx.tz_localize("UTC")
        else:
            idx = idx.tz_convert("UTC")
        out.index = idx

    return out.sort_index()

def to_base_key(col_name: str) -> str:
    """Extrahiert den Teil vor dem ersten Punkt – z.B. 'FL_00352208' aus 'FL_00352208.PLAN.MengeHH'."""
    return str(col_name).split(".", 1)[0].strip().strip('"')

# ================================
# 1) Wetterdaten vorbereiten
# ================================
# Erwartet Spalten: ['timestamp','stat_id','tl','rf','ff','ffx','cglo','so_h','rr','rrm','tb10','tb20']
dfw = df_weather.copy()

# Zeit normalisieren
dfw["timestamp"] = pd.to_datetime(dfw["timestamp"], errors="coerce", utc=True)
dfw = dfw.dropna(subset=["timestamp"])

# Wetterspalten mit Präfix w_
weather_cols_raw = ["tl","rf","ff","ffx","cglo","so_h","rr","rrm","tb10","tb20"]
weather_cols_map = {c: f"w_{c}" for c in weather_cols_raw}
dfw = dfw.rename(columns=weather_cols_map)

# stationweise vorbereiten (Index = timestamp)
weather_by_station = {
    sid: g.drop(columns=["stat_id"]).set_index("timestamp").sort_index()
    for sid, g in dfw.groupby("stat_id", dropna=True)
}

# ================================
# 2) Verbrauchsdaten vorbereiten (df_sub)
# ================================
dfs = ensure_utc_index(df_sub)  # 'von' ist Index → nur normalisieren


# Optional: auf Startzeit beschneiden
START = pd.Timestamp("2015-01-01 06:00:00", tz="UTC")
dfs = dfs.loc[dfs.index >= START]

# ================================
# 3) Mapping Verbrauchs-Key -> stat_id (aus result)
# ================================
# result Spalten: nemesys_key, stat_id, ...
res = result.copy()
res["nemesys_key"] = res["nemesys_key"].astype(str)
key2station = res.dropna(subset=["stat_id"]).set_index("nemesys_key")["stat_id"].to_dict()

# ================================
# 4) Verknüpfung: je Spalte Verbrauch + passende Wetterstation
# ================================
series_with_weather = {}
missing_map = []      # Spalten ohne Mapping (nemesys_key -> stat_id)
missing_station = []  # Spalten mit Mapping, aber ohne Wetterdaten für stat_id

for col in dfs.columns:
    # Verbrauchsserie (als 'consumption' benannt)
    cons = pd.DataFrame({"consumption": dfs[col]})

    base = to_base_key(col)          # z.B. 'FL_00352208'
    sid = key2station.get(base)      # zugehörige stat_id

    if sid is None:
        missing_map.append(col)
        continue

    w = weather_by_station.get(sid)
    if w is None or w.empty:
        missing_station.append((col, sid))
        continue

    # Zeitbasierter Join auf Index (beide UTC)
    merged = cons.join(w, how="left")

    # Option: nur stündliche Frequenz erzwingen (falls gewünscht)
    # merged = merged.asfreq("h")

    # Nützliche Metadaten
    merged.attrs["station_id"] = sid
    merged.attrs["base_key"] = base

    series_with_weather[col] = merged

# ================================
# 5) Kurze Statusmeldungen
# ================================
print(f"Erfolgreich verknüpfte Serien: {len(series_with_weather)}")
if missing_map:
    print(f"Kein Mapping (nemesys_key -> stat_id) für {len(missing_map)} Spalten. Beispiele:", missing_map[:5])
if missing_station:
    print(f"Gemappte Station ohne Wetterdaten für {len(missing_station)} Spalten. Beispiele:", missing_station[:5])

# =================================
# 6) Beispielzugriffe
# =================================
#  - Liste der Keys:
# list(series_with_weather.keys())[:5]

#  - Eine Serie inspizieren:
# k = list(series_with_weather.keys())[0]
# series_with_weather[k].head()

#  - Ein Modell-Feature-Set definieren:
# features_full = ['w_tl','w_rf','w_ff','w_ffx','w_cglo','w_so_h','w_rr','w_rrm','w_tb10','w_tb20']
# X = series_with_weather[k][features_full]
# y = series_with_weather[k]['consumption']


Erfolgreich verknüpfte Serien: 319
Kein Mapping (nemesys_key -> stat_id) für 3 Spalten. Beispiele: ['CV_00003454.PLAN.Menge', 'FL_00342878.PLAN.Menge.2', 'FL_00342878.PLAN.Menge.1']


In [68]:

def to_naive_utc_index(df: pd.DataFrame) -> pd.DataFrame:
    """Index -> datetime, falls tz-aware: nach UTC konvertieren und tz entfernen (naiv)."""
    out = df.copy()
    idx = pd.to_datetime(out.index, errors="coerce")
    # tz-aware -> nach UTC, dann tz entfernen
    if getattr(idx, "tz", None) is not None:
        idx = idx.tz_convert("UTC").tz_localize(None)
    out.index = idx
    return out.sort_index()

# --- ALLE Serien im fertigen Dictionary auf naive UTC setzen ---
for k, dfk in list(series_with_weather.items()):
    series_with_weather[k] = to_naive_utc_index(dfk)

# Jetzt NAIVER Startzeitpunkt -> kein tz-Mismatch mehr
START = pd.Timestamp("2015-01-01 06:00:00")

for k, dfk in list(series_with_weather.items()):
    series_with_weather[k] = dfk.loc[dfk.index >= START]

In [69]:
series_with_weather['FL_00024360.PLAN.MengeHH.2']

,consumption,w_tl,w_rf,w_ff,w_ffx,w_cglo,w_so_h,w_rr,w_rrm,w_tb10,w_tb20
2015-01-01 06:00:00,51.213,-4.0,81.0,1.1,2.4,0.0,0.0,0.0,0.0,0.7,1.7
2015-01-01 07:00:00,51.302,-3.8,81.0,1.0,2.0,1.0,0.0,0.0,2.0,0.7,1.7
2015-01-01 08:00:00,51.568,-3.4,83.0,0.4,1.1,19.0,0.0,0.0,55.0,0.7,1.7
2015-01-01 09:00:00,52.012,-2.6,83.0,0.7,2.4,46.0,0.0,0.0,22.0,0.7,1.7
2015-01-01 10:00:00,51.745,-2.3,84.0,1.1,2.9,45.0,0.0,0.0,20.0,0.7,1.7
...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,47.458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 20:00:00,47.717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 21:00:00,47.114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 22:00:00,47.216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
dfp = df_price.copy()

# Spalte "Datum" in DatetimeIndex umwandeln (naiv wie series_with_weather)
if "Datum" in dfp.columns:
    dfp["Datum"] = pd.to_datetime(dfp["Datum"], errors="coerce")
    dfp = dfp.dropna(subset=["Datum"]).set_index("Datum").sort_index()

# Spalten ggf. umbenennen (z. B. Leerzeichen entfernen)
dfp = dfp.rename(columns=lambda c: c.strip().replace(" ", "_"))

# ================================
# Preisdaten zu allen Serien joinen
# ================================
for k, dfk in list(series_with_weather.items()):
    series_with_weather[k] = dfk.join(dfp, how="left")

series_with_weather['FL_00024360.PLAN.MengeHH.2']

,consumption,w_tl,w_rf,w_ff,w_ffx,w_cglo,w_so_h,w_rr,w_rrm,w_tb10,w_tb20,CEGHIX,TRP_NCG
2015-01-01 06:00:00,51.213,-4.0,81.0,1.1,2.4,0.0,0.0,0.0,0.0,0.7,1.7,23.61,21.50
2015-01-01 07:00:00,51.302,-3.8,81.0,1.0,2.0,1.0,0.0,0.0,2.0,0.7,1.7,23.61,21.50
2015-01-01 08:00:00,51.568,-3.4,83.0,0.4,1.1,19.0,0.0,0.0,55.0,0.7,1.7,23.61,21.50
2015-01-01 09:00:00,52.012,-2.6,83.0,0.7,2.4,46.0,0.0,0.0,22.0,0.7,1.7,23.61,21.50
2015-01-01 10:00:00,51.745,-2.3,84.0,1.1,2.9,45.0,0.0,0.0,20.0,0.7,1.7,23.61,21.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,47.458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.10,23.12
2018-12-31 20:00:00,47.717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.10,23.12
2018-12-31 21:00:00,47.114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.10,23.12
2018-12-31 22:00:00,47.216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.10,23.12


In [71]:
na_overview.head()

,na_ratio,bin,nemesys_key,geom_g,geometry,pt_interior,pt_x,pt_y
FL_00352208.PLAN.MengeKW,1.0,>30%,FL_00352208,MULTIPOLYGON (((453137.430256977 5163557.52722...,"MULTIPOLYGON (((453137.43 5163557.527, 453137....",POINT (449581.692 5173939.372),449581.692322,5.173939e+06
KN_00003623.PLAN.Menge,1.0,>30%,KN_00003623,POINT (629249.710253795 5319336.72648159),POINT (629249.71 5319336.726),POINT (629249.71 5319336.726),629249.710254,5.319337e+06
KN_00003613.PLAN.Menge,1.0,>30%,KN_00003613,POINT (619503.043571605 5315717.57029582),POINT (619503.044 5315717.57),POINT (619503.044 5315717.57),619503.043572,5.315718e+06
KN_00003602.PLAN.Menge,1.0,>30%,KN_00003602,POINT (616169.6906068642 5315623.142645395),POINT (616169.691 5315623.143),POINT (616169.691 5315623.143),616169.690607,5.315623e+06
KN_00003549.PLAN.Menge,1.0,>30%,KN_00003549,POINT (613600.973378887 5316674.7537472),POINT (613600.973 5316674.754),POINT (613600.973 5316674.754),613600.973379,5.316675e+06


In [72]:
from collections import defaultdict

# --- 1) Helfer: sichere Abfrage aus na_overview ---
def get_na_meta(col):
    if col in na_overview.index:
        return {
            "bin": na_overview.loc[col, "bin"],
            "na_ratio": float(na_overview.loc[col, "na_ratio"]),
        }
    return {"bin": pd.NA, "na_ratio": pd.NA}

# --- 2) Metadaten an jedes DF hängen (attrs) ---
for col, dfk in series_with_weather.items():
    meta = get_na_meta(col)
    # vorhandene attrs (z.B. station_id, base_key) beibehalten
    dfk.attrs.update({
        "bin": meta["bin"],
        "na_ratio": meta["na_ratio"],
        # nützlich für Checks:
        "has_weather": any(c.startswith("w_") for c in dfk.columns),
        "has_price": set(["CEGHIX", "TRP_NCG"]).issubset(dfk.columns),
    })
    series_with_weather[col] = dfk  # (explizit, falls du immutables magst)

# --- 3) Zentrales Registry-DataFrame bauen ---
rows = []
for col, dfk in series_with_weather.items():
    rows.append({
        "series_col": col,
        "base_key": dfk.attrs.get("base_key"),
        "station_id": dfk.attrs.get("station_id"),
        "bin": dfk.attrs.get("bin"),
        "na_ratio": dfk.attrs.get("na_ratio"),
        "has_weather": dfk.attrs.get("has_weather"),
        "has_price": dfk.attrs.get("has_price"),
        "start": dfk.index.min(),
        "end": dfk.index.max(),
        "n": len(dfk),
    })

registry = pd.DataFrame(rows).set_index("series_col").sort_values(["bin","na_ratio"], ascending=[True, False])

# --- 4) Praktische Views/Filter ---
def get_series_by_bin(bin_label: str):
    """Dictionary subset aller Serien in einem Bin-Cluster."""
    return {k: v for k, v in series_with_weather.items() if v.attrs.get("bin") == bin_label}

# Beispiel:
cluster_10_15 = get_series_by_bin("10–15%")

In [74]:
type(cluster_10_15)

dict

In [75]:
series_by_bin = defaultdict(dict)
for k, dfk in series_with_weather.items():
    b = dfk.attrs.get("bin") if pd.notna(dfk.attrs.get("bin")) else "unbekannt"
    series_by_bin[b][k] = dfk


In [91]:
import pickle

with open("../data/series_by_bin.pkl", "wb") as f:
    pickle.dump(series_by_bin, f)

# Laden
with open("../data/series_by_bin.pkl", "rb") as f:
    series_by_bin_loaded = pickle.load(f)

In [92]:
series_by_bin_loaded.keys()

dict_keys(['0–5%', '10–15%', '5–10%', '20–25%', '>30%', '25–30%'])

In [93]:
for k, v in series_by_bin_loaded.items():
    print(f"{k}: {len(v)} Serien")

0–5%: 63 Serien
10–15%: 131 Serien
5–10%: 35 Serien
20–25%: 62 Serien
>30%: 27 Serien
25–30%: 1 Serien
